# Q-Learning

#### Rendering Gym in Colab
https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t

In [ ]:
#!pip install gym==0.20.0
#!pip install torch==1.8.0
#!pip install numpy==1.21.2
#!pip install matplotlib==3.4.3
#!pip install gym[box2d]

In [1]:
from collections import defaultdict

import tqdm

import gym

import matplotlib.pyplot as plt
import numpy as np

# Environment

https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

In [4]:
env = gym.make('FrozenLake-v1', 
               desc = None, 
               map_name = "4x4", 
               is_slippery = False)

#env.action_space.seed(123)
#observation = env.reset(seed = 123)
"""
observation = env.reset(seed = 123)

observations = []
rewards = []
N_iters = 1000
for _ in tqdm.tqdm(range(N_iters)):
    action = env.action_space.sample()
    observation, reward, terminated, info = env.step(action)

    observations.append(observation)
    rewards.append(reward)

    if terminated:
        observation = env.reset()
""";
#env.close()

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [23]:
#dir(env)

In [24]:
print("Actions: {}, states: {}.".format(env.action_space, env.observation_space))

Actions: Discrete(4), states: Discrete(16).


# Q-Learning Agent

https://github.com/rasbt/machine-learning-book/blob/main/ch19/ch19.ipynb

In [ ]:
class Agent(object):
    def __init__(self, 
                 env,                   # gym env.
                 learning_rate = 0.01,  # Q-learning learning rate.
                 discount_factor = 0.9, # Q-learning discount factor.
                 epsilon = 0.9,         # Greedy epsilon factor.
                 epsilon_min = 0.1,     # Minimum epsilon value to decay to.
                 epsilon_decay = 0.95): # Epsilon decay rate.
      
        self.learning_rate = learning_rate     # Q-learning learning rate.
        self.discount_factor = discount_factor # Q-learning discount factor.
        self.epsilon = epsilon                 # Greedy epsilon factor.
        self.epsilon_min = epsilon_min         # Minimum epsilon value to decay to.
        self.epsilon_decay = epsilon_decay     # Epsilon decay rate.

        self.env = env
        self.nA = env.action_space.n
        self.q_table = defaultdict(lambda: np.zeros(self.nA))

    def choose_action(self, state):
        if np.random.uniform() < self.epsilon:
            action = np.random.choice(self.nA)
        else:
            q_vals = self.q_table[state]
            perm_actions = np.random.permutation(self.nA)
            q_vals = [q_vals[a] for a in perm_actions]
            perm_q_argmax = np.argmax(q_vals)
            action = perm_actions[perm_q_argmax]
        return action

    def learn(self, transition):
        s, a, r, next_s, done = transition
        q_val = self.q_table[s][a]
        if done:
            q_target = r
        else:
            q_target = r + self.discount_factor * np.max(self.q_table[next_s])

        self.q_table[s][a] = self.q_table[s][a] + self.learning_rate * (q_target - q_val)
        
        self._adjust_epsilon()

    def adjust_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay